# Perkenalan 


   ==============================================
   
   M3

    Nama  : Nugroho Wicaksono
    Batch : FTDS-029

    Proyek ini merupakan Projek tambahan yang mendukung Milestone Project 3 yang fokus pada pembangunan data pipeline untuk menganalisis data penjualan laptop dan PC. Pipeline ini dibuat secara profesional menggunakan Apache Airflow untuk mengorkestrasi seluruh alur kerja data. Data yang telah bersih kemudian diolah dan divisualisasikan dengan Elasticsearch dan Kibana untuk mendapatkan wawasan bisnis yang bermanfaat. Untuk memastikan kualitas data, Great Expectations digunakan untuk memvalidasi data dan memastikan akurasinya. Proyek ini bertujuan untuk mendemonstrasikan proses dari data mentah menjadi informasi yang akurat dan terpercaya.
    
   ==============================================


Proyek: Analisis Harga Laptop & Validasi Data dengan Great Expectations
=======================================================================

Tujuan Proyek:
-----------------
Dalam proyek ini, saya berperan sebagai Data Analyst yang bertugas untuk menganalisis tren harga laptop 
di berbagai platform e-commerce. Fokus utama proyek ini mencakup:

1. Mengidentifikasi Faktor Penentu Harga:
   - Menentukan fitur-fitur utama yang memengaruhi harga laptop seperti:
     - Brand (merek)
     - Jenis prosesor
     - Kapasitas RAM
     - Tipe dan ukuran penyimpanan
     - Ukuran layar
     - Rating dan jumlah ulasan
     - Sistem operasi
   - Tujuannya adalah memahami variasi harga antar model berdasarkan konfigurasi teknis.

2. Pembersihan & Validasi Data (Great Expectations):
   - Dataset mentah umumnya mengandung data yang tidak konsisten, missing value, dan tipe data tidak sesuai.
   - Proses cleaning dilakukan menggunakan DAG pipeline.
   - Validasi data dilakukan dengan library `great_expectations`, meliputi:
     - Pengecekan nilai berada dalam rentang wajar (misalnya: harga > 0)
     - Tipe data kolom sesuai (misalnya: RAM = integer)
     - Kolom nilai unique ada atau tidak
     - Cek duplikasi jika diperlukan
   - Tujuan akhir: memastikan data yang digunakan layak dianalisis dan terpercaya.

3. Memberikan Insight untuk Strategi Bisnis:
   - Mendukung tim e-commerce & bisnis dalam:
     - Menentukan strategi harga yang kompetitif
     - Menyoroti fitur penting di listing produk
     - Meningkatkan tingkat konversi dan kepuasan pelanggan
   - Analisis ini bertujuan membantu perusahaan memahami persepsi nilai oleh konsumen berdasarkan spesifikasi produk.

Tools yang Digunakan:
------------------------
- Python 3.10.18
- pandas 2.3.1
- great_expectations 0.18.19
- PostgreSQL (Docker)
- Jupyter Notebook / VS Code



<div align="center">

| Nama Kolom                  | Tipe Data  |
|----------------------------|------------|
| `id`                       | String     |
| `brand`                    | String     |
| `price`                    | Float      | 
| `currency`                 | String     |
| `color`                    | String     |
| `features`                 | String     |
| `condition`                | String     | 
| `condition_description`    | String     |
| `seller_note`              | String     |
| `gpu`                      | String     |
| `processor`                | String     |
| `processor_speed`          | Float      |      
| `processor_speed_unit`     | String     | 
| `type`                     | String     |
| `width_of_the_display`     | Float      |       
| `height_of_the_display`    | Float      |        
| `os`                       | String     |
| `storage_type`             | String     | 
| `hard_drive_capacity`      | Integer    |     
| `hard_drive_capacity_unit` | String     | 
| `ssd_capacity`             | Integer    | 
| `ssd_capacity_unit`        | String     | 
| `screen_size_inch_`        | Float      | 
| `ram_size`                 | Integer    | 
| `ram_size_unit`            | String     | 


</div>

# A. Libraries

In [1]:
# Import Libraries
import pandas as pd
import great_expectations as gx 
from great_expectations.checkpoint import Checkpoint
from great_expectations.core.batch import RuntimeBatchRequest


In [2]:

print(gx.__version__)

0.18.19


## 1. Load the data (clean version)

In [3]:

# Menggunakan file data dari proyek Anda
df = pd.read_csv('P2M3_Nugroho_Wicaksono_data_clean.csv')

# 2. Load the GE context
context = gx.get_context()

# 3. Add datasource to the context (jika belum ada)
datasource_name = "runtime_pandas_datasource"
if datasource_name not in context.list_datasources():
    context.add_datasource(
        name=datasource_name,
        class_name="Datasource",
        execution_engine={"class_name": "PandasExecutionEngine"},
        data_connectors={
            "default_runtime_data_connector_name": {
                "class_name": "RuntimeDataConnector",
                "batch_identifiers": ["default_identifier_name"]
            }
        })

# 4. Register the dataframe as a runtime batch
batch_request = RuntimeBatchRequest(
    datasource_name="runtime_pandas_datasource",
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="p2m3_nugroho_wicaksono_clean_data",
    runtime_parameters={"batch_data": df},
    batch_identifiers={"default_identifier_name": "default"},
)

# 5. Create or get the expectation suite
expectation_suite_name = "p2m3_nugroho_wicaksono_clean_data_suite"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

# 6. Create validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

# Print validator result
print(validator.head())

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

   id  brand  price currency  color  \
0   1  other  303.8        $   gray   
1   3   dell  175.0        $  black   
2   9   dell  280.0        $  black   
3  14  other  152.0        $   gray   
4  15  other  244.0        $   gray   

                                            features                condition  \
0  Backlit Keyboard,  Built-in Microphone,  Built...                      New   
1  10/100 LAN Card, Backlit Keyboard, Bluetooth, ...                     Used   
2                                              Wi-Fi  Excellent - Refurbished   
3                                              Wi-Fi                      New   
4  Backlit Keyboard, Bluetooth, Built-in Micropho...                      New   

                               condition_description  \
0  A brand-new, unused, unopened, undamaged item ...   
1  An item that has been used previously. The ite...   
2  The item is in like-new condition, backed by a...   
3  A brand-new, unused, unopened, undamaged item ...  

# Validation

## Ekspektasi 1: Kolom ID transaksi harus unik

In [4]:

# Harus ada di dalam validasi
validator.expect_column_values_to_be_unique("id")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 574,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


## Validation Summary: Kolom `id`

### Ekspektasi
Kolom `id` harus unik.

### Hasil Validasi


| Komponen         | Nilai             |
|------------------|-------------------|
| Jumlah Data      | 574               |
| success          | true              |
| unexpected_count | 0                 |
| missing_count    | 0                 |



### Conclusion
Kolom id dalam dataset ini unik dan tidak memiliki nilai yang hilang. Ini mengonfirmasi integritas data dan membuat kolom ini cocok sebagai pengenal unik.



## Ekspektasi 2: Nilai di kolom 'price' harus berada dalam rentang tertentu

In [5]:

# Harus ada di dalam validasi
# Contoh: jumlah pembelian tidak boleh kurang dari 10 dan tidak lebih dari 2348
validator.expect_column_values_to_be_between(
    "price", min_value=10, max_value=2348
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 574,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


## Validation Summary: Kolom `price`

### Ekspektasi
Nilai-nilai di kolom `price` harus berada dalam rentang tertentu, yaitu tidak kurang dari 10 dan tidak lebih dari 2348.

### Hasil Validasi

| Komponen                | Nilai            |
|-------------------------|------------------|
| Jumlah Data             | 574              |
| success                 | true             |
| unexpected_count        | 0                |
| unexpected_percent      | 0.0              |
| partial_unexpected_list | []               |
| missing_count           | 0                |
| raised_exception        | false            |

### Conclusion
Validasi terhadap kolom `price` berhasil sepenuhnya. Semua 574 nilai dalam kolom price berada dalam rentang yang telah ditetapkan (10 hingga 2348) dan tidak ada nilai yang hilang.


## Ekspektasi 3: Isi value set didalam brand harus sesuai

In [6]:

validator.expect_column_values_to_be_in_set(
    "brand",
    value_set=["dell", "hp", "lenovo", "other", "acer", "asus", "microsoft", "samsung", "lg"]
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 574,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


# Validation Summary  
- **Ekspektasi Validasi**: Nilai-nilai di kolom brand harus sesuai dengan set nilai yang telah ditentukan: ["dell", "hp", "Lenovo", "other", "acer", "asus", "microsoft", "samsung", "lg"].

- **Jumlah Elemen**: Sebanyak **574** elemen (baris) divalidasi.

- **Hasil Validasi**: Ekspektasi berhasil terpenuhi (**"success": true**).

- **Analisis Nilai di Luar Set**:

- **unexpected_count**: **0** (Tidak ada nilai di luar set yang ditemukan).

- **unexpected_percent**: **0.0** (Persentase nilai di luar set adalah 0%).

- **partial_unexpected_list**: **[]** (Tidak ada contoh nilai di luar set yang ditemukan).

**Informasi Tambahan**:

- **missing_count**: **0** (Tidak ada nilai yang hilang/kosong pada kolom brand).

- **raised_exception**: **false** (Tidak ada kesalahan teknis yang terjadi).

**Conclusion**  
Validasi terhadap kolom brand berhasil sepenuhnya. Semua nilai brand dalam dataset ini sesuai dengan daftar yang telah ditentukan, dan tidak ada nilai yang hilang. Ini menjamin konsistensi dan integritas data pada kolom kategori brand, yang sangat penting untuk analisis kategori.



## Ekspektasi 4: Tipe data untuk kolom 'price' harus sesuai

In [7]:

validator.expect_column_values_to_be_in_type_list(
    "price", type_list=["int", "float"]
)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Validation Summary  
- **Ekspektasi Validasi**: Tipe data untuk kolom price harus sesuai dengan daftar tipe yang ditentukan, yaitu **int** atau **float**.

- **Hasil Validasi**: Ekspektasi berhasil terpenuhi (**"success": true**).

- **Tipe Data Teramati**: Tipe data yang teramati pada kolom price adalah **"float64"**.

**Informasi Tambahan**:  
- **raised_exception**: **false** (Tidak ada kesalahan teknis yang terjadi).

**Conclusion**  
Validasi terhadap tipe data pada kolom price berhasil. Tipe data yang teramati, yaitu float64, termasuk dalam daftar tipe data yang diharapkan (int atau float). Ini memastikan konsistensi tipe data pada kolom harga, yang penting untuk operasi perhitungan dan analisis numerik.


## Ekspektasi 5: Kolom-kolom kunci harus ada

In [8]:

validator.expect_column_to_exist("id")

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Validation Summary  
- **Ekspektasi Validasi**: Kolom **id** harus ada dalam dataset.

- **Hasil Validasi**: Ekspektasi berhasil terpenuhi (**"success": true**).

**Informasi Tambahan**:  
- Objek result kosong (**{}**) karena ekspektasi ini hanya memeriksa keberadaan kolom.  
- **raised_exception**: **false** (Tidak ada kesalahan teknis yang terjadi).

**Conclusion**  
Validasi terhadap keberadaan kolom id berhasil. Kolom id ditemukan dalam dataset, yang memastikan bahwa data memiliki pengenal unik yang krusial untuk struktur data.


## Ekspektasi 6: Tidak ada nilai null di kolom-kolom penting

In [9]:

validator.expect_column_values_to_not_be_null("price")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 574,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Validation Summary  
- **Ekspektasi Validasi**: Kolom **price** tidak boleh memiliki nilai null (kosong).

- **Jumlah Elemen**: Sebanyak **574** elemen (baris) divalidasi.

- **Hasil Validasi**: Ekspektasi berhasil terpenuhi (**"success": true**).

- **Analisis Nilai Null**:  
- **unexpected_count**: **0** (Tidak ada nilai null yang ditemukan).  
- **unexpected_percent**: **0.0** (Persentase nilai null adalah 0%).  
- **partial_unexpected_list**: **[]** (Tidak ada contoh nilai null yang ditemukan).

**Informasi Tambahan**:  
- **raised_exception**: **false** (Tidak ada kesalahan teknis yang terjadi).

**Conclusion**  
Validasi terhadap kolom price berhasil sepenuhnya. Semua 574 baris pada kolom price memiliki nilai yang valid dan tidak ada yang kosong. Hal ini memastikan kelengkapan data pada kolom harga, yang sangat penting untuk analisis statistik dan perhitungan.


## Ekspektasi 7: Memastikan urutan dan nama kolom sesuai dengan daftar yang diberikan

In [10]:

validator.expect_table_columns_to_match_ordered_list(
        column_list=["id", "brand", "price", "currency", "color", "features", "condition", "condition_description", "seller_note", "gpu", "processor", "processor_speed", "processor_speed_unit", "type", "width_of_the_display", "height_of_the_display", "os", "storage_type", "hard_drive_capacity", "hard_drive_capacity_unit", "ssd_capacity", "ssd_capacity_unit", "screen_size_inch_", "ram_size", "ram_size_unit"]

)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "id",
      "brand",
      "price",
      "currency",
      "color",
      "features",
      "condition",
      "condition_description",
      "seller_note",
      "gpu",
      "processor",
      "processor_speed",
      "processor_speed_unit",
      "type",
      "width_of_the_display",
      "height_of_the_display",
      "os",
      "storage_type",
      "hard_drive_capacity",
      "hard_drive_capacity_unit",
      "ssd_capacity",
      "ssd_capacity_unit",
      "screen_size_inch_",
      "ram_size",
      "ram_size_unit"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Validation Summary  
- **Ekspektasi Validasi**: Skema kolom dalam dataset harus cocok dengan daftar kolom yang ditentukan dan urutannya.

- **Hasil Validasi**: Ekspektasi berhasil terpenuhi (**"success": true**).

- **Daftar Kolom Teramati**: Terdapat **21** kolom yang diamati dalam dataset, sesuai dengan daftar yang diharapkan, dan dalam urutan yang benar.  
Kolom-kolom tersebut meliputi:  
`id`, `brand`, `price`, `currency`, `color`, `features`, `condition_description`, `seller_note`, `gpu`, `processor`, `processor_speed_unit`, `type`, `width_of_the_display`, `height_of_the_display`, `os`, `storage_type`, `hard_drive_capacity`, `hard_drive_capacity_unit`, `ssd_capacity`, `ssd_capacity_unit`, `screen_size_inch`, `ram_size`, `ram_size_unit`.

**Informasi Tambahan**:  
- **raised_exception**: **false** (Tidak ada kesalahan teknis yang terjadi).

**Conclusion**  
Validasi skema tabel berhasil sepenuhnya. Dataset memiliki semua kolom yang diharapkan dalam urutan yang benar, yang mengonfirmasi struktur data yang konsisten. Hal ini memastikan bahwa data dapat diproses dan diintegrasikan dengan sistem lain tanpa masalah skema.


## Ekspektasi 8 : Memastikan jumlah baris berada dalam rentang yang wajar

In [11]:

validator.expect_table_row_count_to_be_between(
    min_value=0, max_value=1000
)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 574
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

# Validation Summary  
- **Ekspektasi Validasi**: Jumlah baris dalam tabel harus berada di antara 0 dan 1000.

- **Hasil Validasi**: Ekspektasi berhasil terpenuhi (**"success": true**).

- **Jumlah Baris Teramati**: **574** baris ditemukan dalam dataset, berada dalam rentang **min_value=0** dan **max_value=1000**.

**Informasi Tambahan**:  
- **raised_exception**: **false** (Tidak ada kesalahan teknis yang terjadi).

**Conclusion**  
Validasi jumlah baris berhasil. Dataset memiliki **574** baris, yang sesuai dengan ekspektasi jumlah baris yang wajar (**0–1000**). Hal ini memastikan bahwa dataset tidak kosong dan tidak terlalu besar, yang merupakan indikator kualitas data yang baik.


## Saving suite

In [12]:

validator.save_expectation_suite(discard_failed_expectations=False)
print(f"\nEkspektasi suite '{expectation_suite_name}' telah disimpan.")


Ekspektasi suite 'p2m3_nugroho_wicaksono_clean_data_suite' telah disimpan.


# Summary of Expectation Suite Execution  

- **Pembuatan Expectation Suite**: Serangkaian ekspektasi telah dibuat untuk menguji kualitas data, meliputi:

  - **Keunikan**: Memastikan kolom **id** unik.
  - **Rentang Nilai**: Memastikan nilai kolom **price** berada dalam rentang yang wajar (**10 hingga 2348**).
  - **Nilai yang Diizinkan**: Memastikan nilai kolom **brand** hanya berasal dari daftar yang telah ditentukan.
  - **Tipe Data**: Memastikan kolom **price** memiliki tipe data numerik (**int** atau **float**).
  - **Keberadaan Kolom**: Memastikan kolom kunci seperti **id** ada.
  - **Tidak Ada Nilai Kosong**: Memastikan kolom **price** tidak memiliki nilai null.
  - **Struktur Kolom**: Memastikan daftar kolom dan urutannya sesuai dengan yang diharapkan.
  - **Jumlah Baris**: Memastikan jumlah baris data berada dalam rentang yang wajar (**0 hingga 1000**).

- **Penyimpanan Expectation Suite**: Semua ekspektasi yang telah dibuat dikumpulkan dalam satu suite dan disimpan.  
  Hal ini memungkinkan penggunaan ulang aturan validasi ini untuk data baru secara otomatis di masa depan.

**Conclusion**  
Kode ini berhasil melaksanakan serangkaian validasi kualitas data yang komprehensif menggunakan **Great Expectations**.  
Semua ekspektasi berhasil **terpenuhi**, menunjukkan bahwa dataset memiliki **kualitas tinggi**:  
data terstruktur dengan baik, **konsisten**, **tanpa nilai yang hilang atau tidak wajar**, dan **siap digunakan** untuk tahap analisis atau pemodelan lebih lanjut.


## Create Checkpoint

In [13]:

# Nama checkpoint diganti agar sesuai dengan proyek Anda
my_checkpoint_name = "p2m3_nugroho_wicaksono_clean_data_checkpoint"
checkpoint = Checkpoint(
    name=my_checkpoint_name,
    run_name_template="%Y%m%d-%H%M%S",
    data_context=context,
    expectation_suite_name=expectation_suite_name,
    action_list=[
        {
            "name": "storing_the_validation_result",
            "action": {"class_name": "StoreValidationResultAction"},
        },
        {"name": "update_the_data_docs", "action": {"class_name": "UpdateDataDocsAction"}},
    ],
)

# 7. Add the checkpoint to the context
context.add_or_update_checkpoint(checkpoint=checkpoint)

# 8. Run the checkpoint
print("Menjalankan checkpoint...")
result = checkpoint.run(batch_request=batch_request)

# 9. Print the result
print("\nCheckpoint Result:")
print(result)

Menjalankan checkpoint...


Calculating Metrics:   0%|          | 0/27 [00:00<?, ?it/s]


Checkpoint Result:
{
  "run_id": {
    "run_name": "20250802-170120",
    "run_time": "2025-08-03T00:01:20.729862+07:00"
  },
  "run_results": {
    "ValidationResultIdentifier::p2m3_nugroho_wicaksono_clean_data_suite/20250802-170120/20250802T170120.729862Z/b74ab132d40cd406cce38ec136c120c9": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_be_unique",
              "kwargs": {
                "column": "id",
                "batch_id": "b74ab132d40cd406cce38ec136c120c9"
              },
              "meta": {}
            },
            "result": {
              "element_count": 574,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "missing_count": 0,
              "missing_percent": 0.0,
              "unexpected_percent_total": 0.0,
 

# Checkpoint Configuration Summary

- **Definisi Checkpoint**:  
  Sebuah **Checkpoint** bernama **p2m3_nugroho_wicaksono_clean_data_checkpoint** didefinisikan.  
  Checkpoint ini berfungsi sebagai **titik pemeriksaan otomatis** yang mengintegrasikan seluruh proses validasi data.

- **Konfigurasi Checkpoint**:
  - **Nama Checkpoint**: `p2m3_nugroho_wicaksono_clean_data_checkpoint`
  - **run_name_template**: Template `%Y%m%d-%H%M%S` digunakan untuk memberi nama unik pada setiap eksekusi, berdasarkan **waktu dan tanggal**.
  - **expectation_suite_name**: Terhubung ke suite ekspektasi yang berisi seluruh aturan validasi.
  - **action_list**:
    - `StoreValidationResultAction`: Menyimpan hasil validasi ke repositori **Great Expectations**.
    - `UpdateDataDocsAction`: Memperbarui **Data Docs** untuk menampilkan hasil validasi terbaru.

- **Eksekusi Checkpoint**:
  - Checkpoint dijalankan dengan `checkpoint.run()`.
  - Aksi `store_validation_result` dan `update_data_docs` dieksekusi secara **otomatis**.
  - Hasil eksekusi disimpan dalam variabel `result`, berisi **status success, statistik, dan metadata lainnya**.

**Conclusion**  
Checkpoint berhasil dikonfigurasi dan dijalankan dengan baik.  
Proses ini menunjukkan praktik terbaik dalam **manajemen kualitas data**, karena validasi dijalankan secara **otomatis**, hasilnya disimpan dan dokumentasi diperbarui.  
Hal ini memastikan proses validasi dapat dilakukan secara **berulang dan terdokumentasi**, mendukung **monitoring kualitas data yang berkelanjutan**.


## Create Documentation

In [15]:
context.build_data_docs()
context.open_data_docs()  # ini membuka browser dengan hasil validasi

# Ringkasan Hasil Validasi Data

Delapan ekspektasi (expectation) telah diterapkan terhadap dataset penjualan laptop untuk memastikan kualitas data. Berikut adalah ringkasan hasilnya:

1. **Unik (Unique)**  
   - Kolom `id` divalidasi untuk memastikan tidak ada duplikasi.  
   - **Hasil**: Validasi berhasil, semua 574 baris memiliki nilai `id` yang unik.

2. **Rentang Nilai (Range Check)**  
   - Kolom `price` harus berada antara **10 hingga 2348**.  
   - **Hasil**: Semua nilai dalam rentang, tidak ada outlier.

3. **Nilai yang Diizinkan (Set Membership)**  
   - Kolom `brand` hanya boleh berisi nilai dari daftar tertentu:  
     `["dell", "hp", "lenovo", "other", "acer", "asus", "microsoft", "samsung", "lg"]`.  
   - **Hasil**: Semua nilai valid dan sesuai set.

4. **Tipe Data (Data Type)**  
   - Kolom `price` harus memiliki tipe data `int` atau `float`.  
   - **Hasil**: Tipe data yang teramati adalah `float64` dan valid.

5. **Keberadaan Kolom (Column Existence)**  
   - Kolom `id` harus ada dalam dataset.  
   - **Hasil**: Kolom ditemukan dan ekspektasi berhasil.

6. **Tidak Ada Nilai Kosong (Not Null)**  
   - Kolom `price` tidak boleh mengandung nilai `null`.  
   - **Hasil**: Semua nilai tersedia, tidak ada missing value.

7. **Struktur Kolom (Schema Match)**  
   - Urutan dan daftar kolom harus cocok dengan yang ditentukan.  
   - **Hasil**: Semua 25 kolom terdeteksi sesuai urutan dan nama.

8. **Jumlah Baris (Row Count Range)**  
   - Jumlah baris dataset harus berada antara **0 dan 1000**.  
   - **Hasil**: Dataset memiliki 574 baris, sesuai ekspektasi.

---

**Kesimpulan Umum**  
Seluruh validasi berhasil dengan status **"success: true"**.  
Dataset tidak mengandung nilai hilang, outlier, atau kesalahan skema.  
Struktur dan isi data sepenuhnya memenuhi standar kualitas, menjadikan dataset ini siap untuk dipakai dalam analisis lanjutan atau pemodelan machine learning.
